In [2]:
import torch
print(torch.__version__)
print(f"CUDA: {torch.cuda.is_available()}")
# GPUの情報を確認
!nvidia-smi

2.7.1+cu128
CUDA: True
Mon Jul  7 20:05:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.64.01              Driver Version: 576.80         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        On  |   00000000:01:00.0  On |                  N/A |
|  0%   43C    P8             27W /  575W |    2575MiB /  32607MiB |      6%      Default |
|                                         |                        |                  N/A |
+------------------------